In [18]:
import pandas as pd
import warnings

import numpy as np


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 65)
pd.set_option('display.max_rows', 140)

In [19]:
#read in the raw dataset
df = pd.read_csv('data_set_10_17.csv')

#a simple function to get info about the dataset
def gradestats(df):
    total = df["STUDENT"].nunique()
    grads = df.groupby("STUDENT")["DEG_CD"].apply(lambda x: x.notnull().any()).sum()
    percent = np.round(grads/total*100,2)
    print('Rows:', len(df))
    print('Total:', total)
    print('Grads:', grads)
    print('Percent:', percent)
    print()

gradestats(df)

Rows: 30394
Total: 13065
Grads: 7622
Percent: 58.34



In [20]:
cols = df.columns.to_list()
cols

['STUDENT',
 'MAJOR_CURR',
 'MAJOR_COLL',
 'CLSFN_YR',
 'ENTRY_SEM_CD',
 'ENTRY_CCYY',
 'LEVEL',
 'SEM_CCYY',
 'SEM_CD',
 'OFFER_DEPT_ABRVN',
 'CRSE',
 'SECT',
 'GRADE_CATGORY',
 'CRSE_TITLE',
 'SEM_CCYY.1',
 'SEM_CD.1',
 'DEG_CD',
 'DEG_DATEDB',
 'MAJOR_CURR.1',
 'DEG_DESCR',
 'LAST_RGST_TERM',
 'CLSFN_YR.1',
 'MAJOR_CURR.2',
 'MAJOR_COLL.1',
 'Workday Enrolled in Fall 2024',
 'Workday Enrolled in Fall 2024 Class Standing',
 'Workday Enrolled in Fall 2024 Primary Program of Study']

In [21]:
#We will only consider students who initially enrolled between 2011 and 2020 inclusive. 
#The reason for this is that 2020 enrollment is the latest that allows for 8 semesters (4 yrs) of courses to be taken
#and earlier than 2011 data may have issues ie courses that are no longer offered, people that got a degree and returned years laters
df = df[df['ENTRY_CCYY'] <= 2020]
df = df[df['ENTRY_CCYY'] >= 2011]

print('After filtering for students who enrolled between 2011 and 2020')
gradestats(df)

#dropped degrees are graduate degrees. kept degrees are undergraduate degrees.
dropped_degrees = ['MRE', 'MCP', 'MED', 'MFA', 'MTR', 'MHC', 'MFS', 'MHO', 'MAA', 'DVM', 'MBS', 'MBA', 'MA', 'MEN', 'MFN', 'MAT', 'PHD', 'MS']

#here I drop the graduate degrees
for x in dropped_degrees:
    df = df.drop(df[df['DEG_CD'] == x].index)

print('After dropping graduate degrees')
gradestats(df)

After filtering for students who enrolled between 2011 and 2020
Rows: 24408
Total: 9870
Grads: 7250
Percent: 73.45

After dropping graduate degrees
Rows: 23248
Total: 9860
Grads: 7240
Percent: 73.43



In [22]:
check = df[['STUDENT',
                'CRSE',
                'SEM_CCYY',
                'SEM_CD'
                ]]
#check how many duplicate rows are there in the dataset
check.duplicated().sum()

np.int64(1431)

In [23]:
#This means that there 1431 rows that are duplicates (in the sense that they correspond to the same course, semester, and student)
#get the indices of the duplicate rows and drop them from the 'df' dataset
duplicates = check[check.duplicated()].index
df.drop(duplicates, inplace=True)

gradestats(df)

Rows: 21817
Total: 9860
Grads: 7240
Percent: 73.43



In [24]:
#print the value counts in the CRSE column
df['CRSE'].value_counts()

CRSE
165     2889
166     2399
265     1679
140     1356
143     1343
104     1228
150     1109
207      972
201      758
317      688
267      676
105      541
301      480
414      475
266      419
160      362
101      335
492      316
341      289
143X     260
385      236
435      212
436      186
30       174
314      172
304      138
373      131
397      122
350      119
25       112
151      112
365      101
415       96
490       85
495       82
181       81
497       76
342       70
302       63
145       47
331       42
481       41
265H      38
407       35
142       34
166H      34
106       33
106X      32
195       32
240       31
182       29
240X      25
424       23
403X      20
202X      20
495X      20
196       19
439       19
504       18
515       18
202       17
502       15
421       14
491       14
511       14
501       12
474X      11
505       10
510       10
408X      10
474        9
441        9
442        7
145X       7
469X       5
422X       5
139X   

In [25]:
#remove the 'X' appearing in any CRSE name
df['CRSE'] = df['CRSE'].str.replace('X', '')

#the courses x = '302' and '403' both refer to abstract algebra II, so let's combine them and call them '302'
#these are supposed to be the same, so re-number the '403' to '302'
df['CRSE'] = df['CRSE'].replace('403', '302')

In [26]:
#Course to be dropped because the content changes year-to-year or they are no longer/not regularly offered
drop_dict = {
            "10": "HIGH SCHOOL ALGEBRA",
            "492": "UNDERGRAD SEMINAR",
            "495": "SPECIAL TOPICS",
            "490": "INDEPENDENT STUDY",
            "106": "DISCOVERING MATH",
            "181": "LIFE SCI CALC&MDL I",
            "331": "TOPOLOGY"
            }

In [27]:
#Next, we format the semester in which the student entered the university, took the course, and obtained a degree
#We will encode the semester as the last two digits of the year + 0.5 for fall and 0 for spring or summer
pairs = [['ENTRY_CCYY','ENTRY_SEM_CD'],
         ['SEM_CCYY','SEM_CD'],
         ['SEM_CCYY.1','SEM_CD.1']]

for ccyy, sem in pairs:
    #get the last two digits of the year
    df[ccyy] = df[ccyy] % 1000

    #encode the semesters as 0.5 for fall and 0 for spring or summer (value=1 in the dataset)
    df[sem] = df[sem].replace({'F': 0.5, '1': 0, 'S': 0}) 

    #combine the year and semester to get a unique identifier for each semester
    df[sem] = df[sem] + df[ccyy]

#Next, we create columns that compute the semester in which the course was taken and the semester in which the student graduated
#create a new column named 'CRSE_SEM' that computes the semester in which the particular course was taken
df['CRSE_SEM'] = (df['SEM_CD'] - df['ENTRY_SEM_CD'])*2 + 1

#create a new column named 'GRAD_CD' that computes the semester in which the student graduated
df['GRAD_SEM'] = (df['SEM_CD.1'] - df['ENTRY_SEM_CD'])*2 + 1

#get a list of students with more than one degree
degs_per_student = df.groupby('STUDENT')['GRAD_SEM'].nunique()
students_mult_degs = degs_per_student[degs_per_student > 1].index.tolist()

#Remove all rows corresponding to the later degrees
for student in students_mult_degs:
    #get the first degree
    first_degree = df[df['STUDENT'] == student]['GRAD_SEM'].min()

    #remove all rows corresponding to the later degrees
    df = df.drop(df[(df['STUDENT'] == student) & (df['GRAD_SEM'] > first_degree)].index)

print('After removing second degrees')
gradestats(df)

#remove all rows corresponding to students that gradated in less than 4 semesters or greater than 12 semesters (don't touch the rows with null values)
df = df.drop(df[(df['GRAD_SEM'] < 4) | (df['GRAD_SEM'] > 12)].index)

print('After filtering for students that graduated in 4 to 12 semesters')
gradestats(df)

#drop all rows for which CRSE_SEM is greater than 8 or greater than GRAD_SEM (if this occurs, it means that the student took the course after they graduated; this scenario likely happened because the student continued taking courses for a second degree)
df = df.drop(df[(df['CRSE_SEM'] > 8) | (df['CRSE_SEM'] > df['GRAD_SEM'])].index)

print('After removing courses taken after graduation or after 8 semesters')
gradestats(df)

#Remove courses with low enrollment, or having course number less than 100
crse_counts = df['CRSE'].value_counts()
min_enrollment = 50
to_remove = crse_counts[crse_counts <= min_enrollment].index.to_list() + list(drop_dict.keys()) + ['25', '30']
df = df[~df.CRSE.isin(to_remove)]

print('After removing courses with low enrollment or numbered < 100')
gradestats(df)

After removing second degrees
Rows: 21772
Total: 9860
Grads: 7240
Percent: 73.43

After filtering for students that graduated in 4 to 12 semesters
Rows: 21321
Total: 9671
Grads: 7051
Percent: 72.91

After removing courses taken after graduation or after 8 semesters
Rows: 20676
Total: 9545
Grads: 6961
Percent: 72.93

After removing courses with low enrollment or numbered < 100
Rows: 19205
Total: 9179
Grads: 6717
Percent: 73.18



In [28]:
#Next, we format the 'GRADE_CATEGORY' columns as follows:
#"Above C-" and "Satisfactory/Pass" are replaced with a 1 and "C- or Below" is replaced with a -1
# df['GRADE_CATGORY'] = df['GRADE_CATGORY'].replace({'C- OR BELOW': -1, 
#                                                            'ABOVE C-': 1, 
#                                                            'Satisfactory/Pass': 1})

df['GRADE_CATGORY'].replace({'C- OR BELOW': -1,
                                    'ABOVE C-': 1,
                                    'Satisfactory/Pass': 1}, inplace=True)

#We add one column for each course and temporarily stick a 0 in each column
courses = df['CRSE'].unique().tolist()
courses.sort()
for x in courses:
    df[x] = 0 

#Now, in each row, if a student has taken a course x, then the value in the column 'x' is replaced with the product of the 'GRADE_CATEGORY' and 'CRSE_SEM' (so it ranges from -8 to 8)
for x in courses:
    df[x][df['CRSE'] == x] = df['GRADE_CATGORY'] * df['CRSE_SEM']

In [29]:
#drop columns that are not needed
df = df.drop(['LEVEL', 'OFFER_DEPT_ABRVN', 'MAJOR_COLL', 'DEG_DESCR', 'DEG_DATEDB', 'Workday Enrolled in Fall 2024', 'Workday Enrolled in Fall 2024 Class Standing', 'Workday Enrolled in Fall 2024 Primary Program of Study', 'MAJOR_CURR', 'MAJOR_CURR.2', 'MAJOR_CURR.1', 'CLSFN_YR.1', 'LAST_RGST_TERM', 'SECT', 'CLSFN_YR', 'MAJOR_COLL.1','CRSE', 'GRADE_CATGORY', 'SEM_CCYY', 'SEM_CD', 'SEM_CCYY.1', 'SEM_CD.1','CRSE_TITLE', 'ENTRY_CCYY', 'ENTRY_SEM_CD','DEG_CD',
 'CRSE_SEM'], axis='columns')

final_cols = df.columns.to_list()
print(len(final_cols))
final_cols

36


['STUDENT',
 'GRAD_SEM',
 '101',
 '104',
 '105',
 '140',
 '143',
 '145',
 '150',
 '151',
 '160',
 '165',
 '166',
 '201',
 '207',
 '240',
 '265',
 '266',
 '267',
 '301',
 '302',
 '304',
 '314',
 '317',
 '341',
 '342',
 '350',
 '365',
 '373',
 '385',
 '397',
 '414',
 '415',
 '435',
 '436',
 '497']

In [30]:
#replace all null values with 0
df = df.fillna(0)

#group by student, choose the max value in each column
df = df.groupby('STUDENT').agg(lambda x: x.loc[x.abs().idxmax()])

#Add a column 'y' whose value is 0 if 'GRAD_SEM' is 0 or greater than 8, and 1 otherwise
df['y'] = 0
df['y'][(df['GRAD_SEM'] > 0) & (df['GRAD_SEM'] <= 8)] = 1

df

,GRAD_SEM,101,104,105,140,143,145,150,151,160,165,166,201,207,240,265,266,267,301,302,304,314,317,341,342,350,365,373,385,397,414,415,435,436,497,y
STUDENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8.0,0,0,0,0,0,0,0,0,0,1,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0.0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7.0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13059,4.0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13060,10.0,0,0,0,-1,0,0,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13061,0.0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
#Below is a dictionary of all the courses that remain after data cleaning
crse_dict = {
                '104': 'INTRO TO PROBABILTY',
                '105': 'INTRO TO MATH IDEA',
                '140': 'COLLEGE ALGEBRA',
                '143': 'PREP FOR CALCULUS',
                '145': 'APPLD TRIGONOMETR',
                '150': 'DISC MATH BUS&SOC S',
                '151': 'BUS & SOC SCI CALC',
                '160': 'SURVEY OF CALCULUS',
                '165': 'CALCULUS I',
                '166': 'CALCULUS II',
                '207': 'MATRCES&LINEAR ALGB',
                '265': 'CALCULUS III',
                '266': 'ELEM DIFF EQUATNS',
                '267': 'DIFF EQ & TRANSFMS',
                '101': 'ORIENTATION IN MATH',
                '201': 'INTRO TO PROOFS',
                '240': 'INVESTM&CREDIT MATH',
                '301': 'ABSTRACT ALGEBRA I',
                '302': 'ABSTRCT ALGEBRA II',
                '304': 'COMBINATORICS',
                '314': 'GRAPH THEORY',
                '317': 'THRY LINEAR ALGEBRA',
                '350': 'NUMBER THEORY',
                '341': 'INTR THY PROBAB&S I',
                '342': 'INTR THY PR&STAT II',
                '365': 'COMPLEX VARIABLES',
                '373': 'INTR SCIENTF CMPTNG',
                '385': 'INTR PART DIFF EQUA',
                '397': 'IV MATH TCH SECDY',
                '414': 'ANALYSIS I',
                '415': 'ANALYSIS II',
                '435': 'GEOMETRY I',
                '436': 'GEOMETRY II',
                '497': 'TEACH SEC SCHL MATH'
            }

In [32]:
#check that the keys in crse_dict match the columns in the dataset
assert set(crse_dict.keys()) == set(courses)

In [33]:
gen_prereqs = {'143':['140'],
               '160':['145'],
               '165':['143','140'],
               '166':['165','143','140'],
               '207':['165','143','140'],
               '265':['166','165','143','140'],
               '266':['265','166','165','143','140'],
               '267':['265','166','165','143','140'],
               }

math_prereqs = {'304':['201'],
                '314':['201'],
                '317':['201'],
                '350':['201'],
                '414':['201'],
                '435':['201'],
                '436':['201'],
                '301':['317','201'],
                # '407':['317','201'],
                '415':['414','201'],
                '302':['301','317','201'],
                '397':['301','317','201'],

}
prereqs = {**gen_prereqs, **math_prereqs}
prereqs


{'143': ['140'],
 '160': ['145'],
 '165': ['143', '140'],
 '166': ['165', '143', '140'],
 '207': ['165', '143', '140'],
 '265': ['166', '165', '143', '140'],
 '266': ['265', '166', '165', '143', '140'],
 '267': ['265', '166', '165', '143', '140'],
 '304': ['201'],
 '314': ['201'],
 '317': ['201'],
 '350': ['201'],
 '414': ['201'],
 '435': ['201'],
 '436': ['201'],
 '301': ['317', '201'],
 '415': ['414', '201'],
 '302': ['301', '317', '201'],
 '397': ['301', '317', '201']}

In [34]:
#We want to examine whether there is any strangeness in the data (namely, if a student has taken the prerequisite course after the course that requires the prerequisite)

#loop over x in prereqs.keys()
#then, loop over each prereq in prereqs[x]
#get the indices of all students for which df[x] > 0 and df[x] < df[prereq]. 
#If the length of the list of indices is greater than 0, then add the student to a list of students who took the prereq after the course
#store the list of students in a dictionary with the key as the tuple (x, prereq)

prereq_snafu = {}
for x in prereqs.keys():
    for prereq in prereqs[x]:
        snafus = df[(df[x] > 0) & (df[x] < df[prereq])].index.to_list()
        if len(snafus) > 0:
            prereq_snafu[(x, prereq)] = snafus

#convert prereq_snafu to a dataframe where the key is the course pair and the value is the list of students who took the prereq after the course.
prereq_snafu_df = pd.DataFrame(prereq_snafu.items(), columns=['course,prereq', 'students'])
#add a column for the number of students who took the prereq after the course
prereq_snafu_df['count'] = prereq_snafu_df['students'].apply(len)

#check
prereq_snafu_df

,"course,prereq",students,count
0,"(143, 140)","[586, 5188, 5512, 8116, 10651, 11624]",6
1,"(165, 143)","[1760, 4296, 9597, 12865]",4
2,"(165, 140)","[193, 389, 1008, 2921, 3195, 8286, 9561, 10581...",10
3,"(166, 165)","[926, 1403, 2001, 3680, 3698, 4477, 5355, 7701...",9
4,"(166, 143)","[1760, 4296, 10674, 12865]",4
5,"(166, 140)","[389, 10581, 10674]",3
6,"(207, 165)",[10721],1
7,"(207, 143)",[4296],1
8,"(207, 140)",[389],1
9,"(265, 166)","[3031, 3786, 5318, 6418, 11290]",5


In [35]:
snafu_students = prereq_snafu_df['students'].sum()
snafu_students.sort()
# snafu_students = list(set(snafu_students))
print(len(snafu_students))
snafu_students

159


[86,
 193,
 193,
 193,
 389,
 389,
 389,
 586,
 629,
 723,
 926,
 1008,
 1107,
 1107,
 1107,
 1107,
 1403,
 1428,
 1470,
 1532,
 1759,
 1759,
 1759,
 1760,
 1760,
 1760,
 1794,
 1824,
 2001,
 2001,
 2399,
 2574,
 2574,
 2574,
 2652,
 2814,
 2921,
 2951,
 2951,
 2951,
 3031,
 3031,
 3177,
 3195,
 3283,
 3382,
 3583,
 3583,
 3588,
 3680,
 3698,
 3698,
 3786,
 3787,
 3836,
 4086,
 4125,
 4296,
 4296,
 4296,
 4384,
 4384,
 4477,
 4477,
 4749,
 4775,
 5087,
 5128,
 5128,
 5153,
 5188,
 5255,
 5318,
 5318,
 5355,
 5355,
 5512,
 5587,
 6343,
 6418,
 6419,
 6560,
 6935,
 7009,
 7025,
 7025,
 7357,
 7357,
 7670,
 7701,
 7701,
 7819,
 7827,
 7969,
 8092,
 8108,
 8108,
 8108,
 8108,
 8108,
 8108,
 8116,
 8278,
 8286,
 8313,
 8313,
 8313,
 8356,
 8494,
 8518,
 8518,
 8529,
 8692,
 8878,
 9080,
 9318,
 9385,
 9446,
 9451,
 9561,
 9597,
 9644,
 9686,
 9686,
 9686,
 9686,
 9736,
 9885,
 9909,
 10289,
 10581,
 10581,
 10581,
 10651,
 10674,
 10674,
 10674,
 10721,
 11043,
 11043,
 11143,
 11143,
 1129

In [36]:
snafu_students = list(set(snafu_students)) #remove duplicates
snafu_students.sort()
print(f'There are {len(snafu_students)} students who took a course before the prerequisite course:')
df.loc[snafu_students]

There are 112 students who took a course before the prerequisite course:


,GRAD_SEM,101,104,105,140,143,145,150,151,160,165,166,201,207,240,265,266,267,301,302,304,314,317,341,342,350,365,373,385,397,414,415,435,436,497,y
STUDENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
86,4.0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,3,0,0,4,1,2,0,0,0,0,1,4,4,0,3,2,0,1
193,8.0,5,0,0,2,0,0,0,0,0,1,2,6,0,0,3,0,4,6,8,0,0,5,6,0,0,0,7,0,0,5,0,7,0,0,1
389,0.0,0,0,0,7,0,0,1,0,0,2,3,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
586,8.0,0,0,0,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
629,10.0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
723,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,5,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0
926,9.0,3,0,0,0,0,0,0,0,0,3,2,-4,0,0,3,0,6,0,0,0,8,-7,0,0,0,0,0,0,0,8,0,0,0,0,0
1008,8.0,0,0,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1107,8.0,0,0,0,0,0,0,0,0,0,1,2,7,5,0,3,4,0,6,0,7,6,6,0,0,0,0,0,0,0,6,8,0,0,0,1


In [37]:
#Need to fogure out how to deal with these. For the mooment, to keep things simple, we will remove these students from the dataset
# df = df.drop(snafu_students)

In [38]:
df

,GRAD_SEM,101,104,105,140,143,145,150,151,160,165,166,201,207,240,265,266,267,301,302,304,314,317,341,342,350,365,373,385,397,414,415,435,436,497,y
STUDENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,8.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8.0,0,0,0,0,0,0,0,0,0,1,2,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,0.0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7.0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13059,4.0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13060,10.0,0,0,0,-1,0,0,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13061,0.0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
df['y'].value_counts()

y
1    4953
0    4226
Name: count, dtype: int64

In [40]:
df['GRAD_SEM'].value_counts().sort_index()

GRAD_SEM
0.0     2462
4.0      169
5.0      191
6.0      588
7.0      690
8.0     3315
9.0      788
10.0     687
11.0     174
12.0     115
Name: count, dtype: int64

In [41]:
# #save the grouped dataset
df.to_csv('../Data/final_dataset_Nov_12.csv')

In [ ]:
df.apply(lambda x: min(1,x))